# Libraries

In [16]:
import pandas as pd
import psycopg2 as ps2
import re
from os import walk


In [26]:
negative_urls_file = "/usr/share/databases/IMDbReviews/test/urls_pos.txt"

negative_urls = open(negative_urls_file)

In [27]:
def parseQueryResults(curr, columns, query, new_row):
    curr.execute(query)
    
    successful = True
    try:
        result = curr.fetchall()[0]
    except:
        print("MISSING TITLE")
        successful = False
        
    if(successful == True):  
        hmr = 0 
        for elem in result:
            new_row[columns[hmr]] = elem
            hmr += 1
    return(successful)
    
    

In [28]:
def getReview(rid, files, parent_directory):
    review_and_score = {}
    for file in files:
        re_pattern = str(rid) + "_"
        re_pattern_final = "^%s" % (re_pattern)
        is_match = re.search(re_pattern_final, file)
        if(is_match != None):
            #print(file)
            full_file = parent_directory + "/" + file
            review = open(full_file, mode = 'r')
            whole_review = review.read()
            review.close()
            #print(whole_review)
            review_score = file.split("_")[1]
            review_score = review_score.split(".")[0]
            #print("Review Score: ", review_score)
            
            review_and_score = {"Review": whole_review, "Score": review_score}
            
    return(review_and_score)
        
    
    

In [20]:
neg_parent_directory = "/usr/share/databases/IMDbReviews/test/pos"
neg_review_files = []

for(dirpath, dirnames, filenames) in walk(neg_parent_directory):
    neg_review_files.extend(filenames)



In [29]:
neg_parent_directory = "/usr/share/databases/IMDbReviews/test/pos"
neg_review_files = []

for(dirpath, dirnames, filenames) in walk(neg_parent_directory):
    neg_review_files.extend(filenames)

conn = ps2.connect("dbname=imdb user=tmccormack")
curr = conn.cursor()

# files with negative reviews
negative_urls_file = "/usr/share/databases/IMDbReviews/test/urls_pos.txt"
negative_urls = open(negative_urls_file)

review_id = 0
movie_reviews = []

# collect the title id from each negative review
for url in negative_urls:
    # extract the title id from the url
    start = url.find("title/") + len("title/")
    end = url.find("/usercomments")
    tid = url[start:end]
    
    # make a new row 
    new_row = {"ReviewID": review_id}  
    
    #print("TID: ", tid)
    
    # collect information from title_basics table
    title_basics_columns = ["titleid", "titletype", "primarytitle", "originaltitle", "isadult", "startyear", "endyear", "runtimeminutes", "genres"]
    title_basics_query = "SELECT * FROM title_basics_ml WHERE titleid = '%s'" % (tid)
    
    successful_query = parseQueryResults(curr, title_basics_columns, title_basics_query, new_row)
    
    if(successful_query == False):
        break
    else:   
        # get the review from the url
        review_and_score = getReview(review_id, neg_review_files, neg_parent_directory)
        #print("New Row: ",new_row)

        #print(review_and_score)
        for key, val in review_and_score.items():
            #print("Key: ", key)
            #print("Val: ", val)
            new_row[key] = val


        movie_reviews.append(new_row)
        # increment the review id
        review_id += 1

        #print("NR: ", new_row)

    

curr.close()
conn.close()

MISSING TITLE


In [30]:
negative_reviews = pd.DataFrame(movie_reviews)

In [31]:
negative_reviews.head()

,ReviewID,titleid,titletype,primarytitle,originaltitle,isadult,startyear,endyear,runtimeminutes,genres,Review,Score
0,0,tt0406816,movie,The Guardian,The Guardian,False,2006,None,139,"['Action', 'Adventure', 'Drama']",I went and saw this movie last night after bei...,10
1,1,tt0406816,movie,The Guardian,The Guardian,False,2006,None,139,"['Action', 'Adventure', 'Drama']",My boyfriend and I went to watch The Guardian....,10
2,2,tt0406816,movie,The Guardian,The Guardian,False,2006,None,139,"['Action', 'Adventure', 'Drama']",My yardstick for measuring a movie's watch-abi...,7
3,3,tt0406816,movie,The Guardian,The Guardian,False,2006,None,139,"['Action', 'Adventure', 'Drama']",How many movies are there that you can think o...,7
4,4,tt0406816,movie,The Guardian,The Guardian,False,2006,None,139,"['Action', 'Adventure', 'Drama']",This movie was sadly under-promoted but proved...,10


In [32]:
negative_reviews.to_csv("positive_test.csv")

In [25]:
hmr = 0
for elem in negative_urls:
    hmr += 1
print("HMR: ", hmr)

HMR:  12484


In [32]:
movie_reviews

[{'ReviewID': 0,
  'titleid': 'tt0064354',
  'titletype': 'movie',
  'primarytitle': 'Futz',
  'originaltitle': 'Futz',
  'isadult': False,
  'startyear': 1969,
  'endyear': None,
  'runtimeminutes': 92,
  'genres': ["'Comedy'"]}]

In [17]:
pd.DataFrame(movie_reviews)

,ReviewID,titleid,titletype,primarytitle,originaltitle,isadult,startyear,endyear,runtimeminutes,genres
0,0,tt0064354,movie,Futz,Futz,False,1969,None,92,['Comedy']
